In [ ]:
# Check and install required packages
import sys
import subprocess

def install_package(package):
    """Install package if not available"""
    try:
        __import__(package)
        print(f"✅ {package} is already installed")
    except ImportError:
        print(f"🔄 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installed successfully")

# Check critical packages
packages = ['cv2', 'numpy', 'sklearn', 'matplotlib']
package_names = ['opencv-python', 'numpy', 'scikit-learn', 'matplotlib']

for pkg, name in zip(packages, package_names):
    try:
        __import__(pkg)
        print(f"✅ {name} is available")
    except ImportError:
        print(f"❌ {name} is missing")
        print(f"Please run: pip install {name}")

✅ opencv-python is available
✅ numpy is available
✅ scikit-learn is available
✅ matplotlib is available


In [4]:
# Import libraries with error handling and Windows compatibility fixes
import os
import warnings
warnings.filterwarnings('ignore')

# Fix for Windows joblib/scikit-learn CPU detection issue
os.environ['LOKY_MAX_CPU_COUNT'] = '4'  # Set a reasonable default
os.environ['OMP_NUM_THREADS'] = '1'     # Prevent OpenMP conflicts

try:
    import cv2
    print(f"✅ OpenCV version: {cv2.__version__}")
except ImportError as e:
    print(f"❌ OpenCV import error: {e}")
    print("Install with: pip install opencv-python")
    raise

try:
    import numpy as np
    print(f"✅ NumPy version: {np.__version__}")
except ImportError as e:
    print(f"❌ NumPy import error: {e}")
    raise

try:
    # Import with Windows compatibility fix
    import joblib
    joblib.parallel.cpu_count = lambda: 4  # Override CPU detection
    
    from sklearn.cluster import KMeans
    print("✅ Scikit-learn imported (Windows compatibility mode)")
except ImportError as e:
    print(f"❌ Scikit-learn import error: {e}")
    print("Install with: pip install scikit-learn")
    raise
except Exception as e:
    print(f"⚠️ Scikit-learn imported with warnings: {e}")
    from sklearn.cluster import KMeans
    print("✅ Scikit-learn imported (fallback mode)")

import time
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt

print("\\n🎉 All libraries imported successfully with Windows compatibility fixes!")


✅ OpenCV version: 4.12.0
✅ NumPy version: 2.2.0
✅ Scikit-learn imported (Windows compatibility mode)
\n🎉 All libraries imported successfully with Windows compatibility fixes!


In [5]:
# Test camera availability with detailed diagnostics
def test_camera_detailed():
    """Comprehensive camera testing"""
    print("🔍 Testing camera availability...")
    
    available_cameras = []
    
    for i in range(5):  # Test cameras 0-4
        try:
            print(f"Testing camera {i}...", end=" ")
            cap = cv2.VideoCapture(i)
            
            if cap.isOpened():
                # Try to read a frame
                ret, frame = cap.read()
                if ret and frame is not None:
                    height, width = frame.shape[:2]
                    print(f"✅ Available - Resolution: {width}x{height}")
                    available_cameras.append({
                        'index': i,
                        'width': width,
                        'height': height
                    })
                else:
                    print("❌ Can't read frames")
                cap.release()
            else:
                print("❌ Can't open")
                
        except Exception as e:
            print(f"❌ Error: {e}")
    
    if available_cameras:
        print(f"\\n✅ Found {len(available_cameras)} working camera(s)")
        for cam in available_cameras:
            print(f"  Camera {cam['index']}: {cam['width']}x{cam['height']}")
        return available_cameras[0]['index']  # Return first available
    else:
        print("\\n❌ No working cameras found!")
        print("\\n🔧 Troubleshooting:")
        print("  1. Check if camera is connected")
        print("  2. Close other applications using the camera")
        print("  3. Try external USB camera")
        print("  4. Restart your computer")
        print("  5. Check camera drivers")
        return None

# Test cameras
camera_index = test_camera_detailed()


🔍 Testing camera availability...
Testing camera 0... ✅ Available - Resolution: 640x480
Testing camera 1... ❌ Can't open
Testing camera 2... ❌ Can't open
Testing camera 3... ❌ Can't open
Testing camera 4... ❌ Can't open
\n✅ Found 1 working camera(s)
  Camera 0: 640x480


In [6]:
# Simplified color detection class
class SimpleColorDetector:
    """Simplified, robust color detector"""
    
    def __init__(self, camera_index=0):
        self.camera_index = camera_index
        self.cap = None
        self.is_running = False
        
        # Basic color database
        self.colors = {
            'Red': [255, 0, 0], 'Green': [0, 255, 0], 'Blue': [0, 0, 255],
            'Yellow': [255, 255, 0], 'Cyan': [0, 255, 255], 'Magenta': [255, 0, 255],
            'White': [255, 255, 255], 'Black': [0, 0, 0], 'Gray': [128, 128, 128],
            'Orange': [255, 165, 0], 'Purple': [128, 0, 128], 'Pink': [255, 192, 203],
            'Brown': [165, 42, 42], 'Navy': [0, 0, 128], 'Maroon': [128, 0, 0]
        }
    
    def get_closest_color(self, rgb):
        """Find closest color name"""
        min_dist = float('inf')
        closest = 'Unknown'
        
        for name, color_rgb in self.colors.items():
            dist = sum((a - b) ** 2 for a, b in zip(rgb, color_rgb)) ** 0.5
            if dist < min_dist:
                min_dist = dist
                closest = name
        
        return closest
    
    def extract_colors(self, image, n_colors=3):
        """Extract dominant colors using K-means with Windows compatibility"""
        try:
            # Resize for speed
            small_image = cv2.resize(image, (100, 100))
            pixels = small_image.reshape(-1, 3)
            
            # Sample pixels if too many (for performance)
            if len(pixels) > 5000:
                indices = np.random.choice(len(pixels), 5000, replace=False)
                pixels = pixels[indices]
            
            # K-means clustering with Windows-safe parameters
            kmeans = KMeans(
                n_clusters=n_colors, 
                random_state=42, 
                n_init=5,  # Reduced iterations for performance
                max_iter=50,  # Limit max iterations
                algorithm='elkan'  # More stable algorithm
            )
            kmeans.fit(pixels)
            
            colors = kmeans.cluster_centers_.astype(int)
            
            # Calculate percentages
            labels = kmeans.labels_
            total = len(labels)
            percentages = [(np.sum(labels == i) / total * 100) for i in range(n_colors)]
            
            # Sort by percentage
            sorted_data = sorted(zip(colors, percentages), key=lambda x: x[1], reverse=True)
            colors, percentages = zip(*sorted_data)
            
            return list(colors), list(percentages)
            
        except Exception as e:
            print(f"Error in color extraction: {e}")
            # Fallback to simple color averaging if K-means fails
            try:
                return self.extract_colors_fallback(image, n_colors)
            except:
                return [], []
    
    def extract_colors_fallback(self, image, n_colors=3):
        """Fallback color extraction without K-means"""
        try:
            # Simple grid-based color extraction
            height, width = image.shape[:2]
            grid_size = int(np.sqrt(n_colors * 4))  # Create a grid
            
            colors = []
            for i in range(0, height, height // grid_size):
                for j in range(0, width, width // grid_size):
                    if len(colors) >= n_colors:
                        break
                    # Get average color in this grid cell
                    cell = image[i:i+height//grid_size, j:j+width//grid_size]
                    if cell.size > 0:
                        avg_color = np.mean(cell.reshape(-1, 3), axis=0).astype(int)
                        colors.append(avg_color)
                if len(colors) >= n_colors:
                    break
            
            # Equal percentages for fallback
            percentages = [100.0 / len(colors)] * len(colors)
            
            return colors[:n_colors], percentages[:n_colors]
            
        except Exception as e:
            print(f"Fallback color extraction also failed: {e}")
            return [], []
    
    def draw_overlay(self, frame, colors, percentages):
        """Draw color information on frame"""
        if not colors:
            return frame
        
        height, width = frame.shape[:2]
        
        # Background rectangle
        cv2.rectangle(frame, (10, 10), (350, 120), (0, 0, 0), -1)
        cv2.rectangle(frame, (10, 10), (350, 120), (255, 255, 255), 2)
        
        # Title
        cv2.putText(frame, "Live Color Detection", (20, 35), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Draw colors
        y_pos = 55
        for i, (color, percent) in enumerate(zip(colors[:3], percentages[:3])):
            # Color rectangle (BGR for OpenCV)
            color_bgr = (int(color[2]), int(color[1]), int(color[0]))
            cv2.rectangle(frame, (20, y_pos - 8), (45, y_pos + 8), color_bgr, -1)
            cv2.rectangle(frame, (20, y_pos - 8), (45, y_pos + 8), (255, 255, 255), 1)
            
            # Color name and percentage
            color_name = self.get_closest_color(color)
            text = f"{color_name}: {percent:.1f}%"
            cv2.putText(frame, text, (55, y_pos + 3), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            y_pos += 20
        
        # Instructions
        cv2.putText(frame, "Press 'q' to quit", (20, height - 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return frame
    
    def run_detection(self, duration=None):
        """Run live color detection"""
        if self.camera_index is None:
            print("❌ No camera available for detection")
            return
        
        try:
            # Initialize camera
            self.cap = cv2.VideoCapture(self.camera_index)
            if not self.cap.isOpened():
                print(f"❌ Could not open camera {self.camera_index}")
                return
            
            print("🚀 Starting live color detection...")
            print("📋 Controls: Press 'q' to quit")
            
            self.is_running = True
            start_time = time.time()
            frame_count = 0
            
            while self.is_running:
                ret, frame = self.cap.read()
                if not ret:
                    print("❌ Failed to read from camera")
                    break
                
                frame_count += 1
                
                # Analyze colors every 5 frames for performance
                if frame_count % 5 == 0:
                    # Convert BGR to RGB for analysis
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    colors, percentages = self.extract_colors(frame_rgb)
                    
                    # Draw overlay (frame is in BGR for display)
                    frame = self.draw_overlay(frame, colors, percentages)
                
                # Display frame
                cv2.imshow('Live Color Detection', frame)
                
                # Check for quit
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                
                # Check duration limit
                if duration and (time.time() - start_time) > duration:
                    print(f"⏰ {duration}-second session completed")
                    break
            
            print(f"📊 Processed {frame_count} frames")
            
        except Exception as e:
            print(f"❌ Error during detection: {e}")
            
        finally:
            # Cleanup
            self.is_running = False
            if self.cap:
                self.cap.release()
            cv2.destroyAllWindows()
            print("🧹 Cleanup completed")

print("✅ SimpleColorDetector class loaded!")


✅ SimpleColorDetector class loaded!


In [7]:
# Quick camera test
def quick_camera_test():
    """Quick test to verify camera functionality"""
    if camera_index is None:
        print("❌ No camera available for testing")
        return False
    
    try:
        print(f"🧪 Testing camera {camera_index}...")
        cap = cv2.VideoCapture(camera_index)
        
        if not cap.isOpened():
            print("❌ Camera test failed - cannot open")
            return False
        
        # Try to read a few frames
        for i in range(5):
            ret, frame = cap.read()
            if not ret:
                print(f"❌ Camera test failed - cannot read frame {i+1}")
                cap.release()
                return False
        
        cap.release()
        print("✅ Camera test passed!")
        return True
        
    except Exception as e:
        print(f"❌ Camera test error: {e}")
        return False

# Run camera test
camera_works = quick_camera_test()

if camera_works:
    print("\\n🎯 Ready for live color detection!")
else:
    print("\\n⚠️ Camera issues detected. Please check troubleshooting tips above.")


🧪 Testing camera 0...
✅ Camera test passed!
\n🎯 Ready for live color detection!


In [8]:
# START LIVE COLOR DETECTION
# Run this cell to start live color detection

if camera_index is not None and camera_works:
    print("🚀 STARTING LIVE COLOR DETECTION")
    print("="*40)
    print("📋 Instructions:")
    print("  • Point camera at colored objects")
    print("  • Press 'q' in the video window to quit")
    print("  • Make sure lighting is good")
    print("="*40)
    
    # Create and run detector
    detector = SimpleColorDetector(camera_index)
    detector.run_detection()
    
    print("\\n✅ Live detection session completed!")
    
else:
    print("❌ Cannot start live detection:")
    if camera_index is None:
        print("  • No camera available")
    if not camera_works:
        print("  • Camera not working properly")
    
    print("\\n🔧 Try these solutions:")
    print("  1. Connect a webcam or enable built-in camera")
    print("  2. Close other applications using the camera")
    print("  3. Restart this notebook")
    print("  4. Try running cells individually to isolate issues")


🚀 STARTING LIVE COLOR DETECTION
📋 Instructions:
  • Point camera at colored objects
  • Press 'q' in the video window to quit
  • Make sure lighting is good
🚀 Starting live color detection...
📋 Controls: Press 'q' to quit
📊 Processed 108 frames
🧹 Cleanup completed
\n✅ Live detection session completed!


In [9]:
# Alternative: Quick 10-second test
def quick_test(duration=10):
    """Quick color detection test"""
    if camera_index is not None:
        print(f"⏱️ Running {duration}-second color detection test...")
        detector = SimpleColorDetector(camera_index)
        detector.run_detection(duration=duration)
    else:
        print("❌ No camera available for quick test")

# Uncomment the line below to run a quick 10-second test:
# quick_test(10)

print("💡 Uncomment the line above to run a quick 10-second test")


💡 Uncomment the line above to run a quick 10-second test


In [10]:
# Alternative simple detector (no scikit-learn dependency)
class NoMLColorDetector:
    """Simple color detector without machine learning dependencies"""
    
    def __init__(self, camera_index=0):
        self.camera_index = camera_index
        self.cap = None
        self.is_running = False
        
        # Basic color database
        self.colors = {
            'Red': [255, 0, 0], 'Green': [0, 255, 0], 'Blue': [0, 0, 255],
            'Yellow': [255, 255, 0], 'Cyan': [0, 255, 255], 'Magenta': [255, 0, 255],
            'White': [255, 255, 255], 'Black': [0, 0, 0], 'Gray': [128, 128, 128],
            'Orange': [255, 165, 0], 'Purple': [128, 0, 128], 'Pink': [255, 192, 203],
            'Brown': [165, 42, 42], 'Navy': [0, 0, 128], 'Maroon': [128, 0, 0]
        }
    
    def get_closest_color(self, rgb):
        """Find closest color name"""
        min_dist = float('inf')
        closest = 'Unknown'
        
        for name, color_rgb in self.colors.items():
            dist = sum((a - b) ** 2 for a, b in zip(rgb, color_rgb)) ** 0.5
            if dist < min_dist:
                min_dist = dist
                closest = name
        
        return closest
    
    def extract_colors_simple(self, image, n_colors=3):
        """Extract colors using simple averaging method"""
        try:
            # Resize for speed
            small_image = cv2.resize(image, (50, 50))
            height, width = small_image.shape[:2]
            
            # Divide image into regions
            colors = []
            percentages = []
            
            # Top region
            top_region = small_image[0:height//3, :]
            avg_color = np.mean(top_region.reshape(-1, 3), axis=0).astype(int)
            colors.append(avg_color)
            
            # Middle region  
            mid_region = small_image[height//3:2*height//3, :]
            avg_color = np.mean(mid_region.reshape(-1, 3), axis=0).astype(int)
            colors.append(avg_color)
            
            # Bottom region
            bottom_region = small_image[2*height//3:, :]
            avg_color = np.mean(bottom_region.reshape(-1, 3), axis=0).astype(int)
            colors.append(avg_color)
            
            # Calculate simple percentages (equal for now)
            percentages = [100.0 / len(colors)] * len(colors)
            
            return colors[:n_colors], percentages[:n_colors]
            
        except Exception as e:
            print(f"Error in simple color extraction: {e}")
            return [], []
    
    def draw_overlay(self, frame, colors, percentages):
        """Draw color information on frame"""
        if not colors:
            return frame
        
        height, width = frame.shape[:2]
        
        # Background rectangle
        cv2.rectangle(frame, (10, 10), (350, 120), (0, 0, 0), -1)
        cv2.rectangle(frame, (10, 10), (350, 120), (255, 255, 255), 2)
        
        # Title
        cv2.putText(frame, "Simple Color Detection", (20, 35), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Draw colors
        y_pos = 55
        for i, (color, percent) in enumerate(zip(colors[:3], percentages[:3])):
            # Color rectangle (BGR for OpenCV)
            color_bgr = (int(color[2]), int(color[1]), int(color[0]))
            cv2.rectangle(frame, (20, y_pos - 8), (45, y_pos + 8), color_bgr, -1)
            cv2.rectangle(frame, (20, y_pos - 8), (45, y_pos + 8), (255, 255, 255), 1)
            
            # Color name and percentage
            color_name = self.get_closest_color(color)
            text = f"{color_name}: {percent:.1f}%"
            cv2.putText(frame, text, (55, y_pos + 3), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            y_pos += 20
        
        # Instructions
        cv2.putText(frame, "Press 'q' to quit (No ML Mode)", (20, height - 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return frame
    
    def run_detection(self, duration=None):
        """Run simple color detection"""
        if self.camera_index is None:
            print("❌ No camera available for detection")
            return
        
        try:
            # Initialize camera
            self.cap = cv2.VideoCapture(self.camera_index)
            if not self.cap.isOpened():
                print(f"❌ Could not open camera {self.camera_index}")
                return
            
            print("🚀 Starting simple color detection (no ML dependencies)...")
            print("📋 Controls: Press 'q' to quit")
            
            self.is_running = True
            start_time = time.time()
            frame_count = 0
            
            while self.is_running:
                ret, frame = self.cap.read()
                if not ret:
                    print("❌ Failed to read from camera")
                    break
                
                frame_count += 1
                
                # Analyze colors every 5 frames for performance
                if frame_count % 5 == 0:
                    # Convert BGR to RGB for analysis
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    colors, percentages = self.extract_colors_simple(frame_rgb)
                    
                    # Draw overlay (frame is in BGR for display)
                    frame = self.draw_overlay(frame, colors, percentages)
                
                # Display frame
                cv2.imshow('Simple Color Detection', frame)
                
                # Check for quit
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                
                # Check duration limit
                if duration and (time.time() - start_time) > duration:
                    print(f"⏰ {duration}-second session completed")
                    break
            
            print(f"📊 Processed {frame_count} frames")
            
        except Exception as e:
            print(f"❌ Error during detection: {e}")
            
        finally:
            # Cleanup
            self.is_running = False
            if self.cap:
                self.cap.release()
            cv2.destroyAllWindows()
            print("🧹 Cleanup completed")

# Function to run the alternative detector
def run_simple_detection():
    """Run simple detection without ML dependencies"""
    if camera_index is not None:
        print("🚀 STARTING SIMPLE COLOR DETECTION (No ML)")
        print("="*45)
        print("📋 This version doesn't use scikit-learn")
        print("📋 Press 'q' in the video window to quit")
        print("="*45)
        
        detector = NoMLColorDetector(camera_index)
        detector.run_detection()
        
        print("\\n✅ Simple detection session completed!")
    else:
        print("❌ No camera available for simple detection")

print("✅ Alternative simple detector loaded!")
print("💡 Use run_simple_detection() if scikit-learn issues persist")


✅ Alternative simple detector loaded!
💡 Use run_simple_detection() if scikit-learn issues persist


In [11]:
# Functions to test static images

def test_image_colors(image_path, use_ml=True):
    """
    Test color detection on a static image
    
    Args:
        image_path (str): Path to the image file
        use_ml (bool): Whether to use K-means (True) or simple method (False)
    """
    try:
        # Load image
        print(f"📁 Loading image: {image_path}")
        image = cv2.imread(image_path)
        if image is None:
            print(f"❌ Could not load image from {image_path}")
            return None
        
        # Convert BGR to RGB for processing
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        print(f"✅ Image loaded - Size: {image_rgb.shape[1]}x{image_rgb.shape[0]}")
        
        # Extract colors
        if use_ml:
            try:
                detector = SimpleColorDetector(0)
                colors, percentages = detector.extract_colors(image_rgb, n_colors=5)
                print("🎨 Used K-means clustering for color extraction")
            except Exception as e:
                print(f"⚠️ K-means failed: {e}, falling back to simple method")
                detector = NoMLColorDetector(0)
                colors, percentages = detector.extract_colors_simple(image_rgb, n_colors=3)
        else:
            detector = NoMLColorDetector(0)
            colors, percentages = detector.extract_colors_simple(image_rgb, n_colors=3)
            print("🎨 Used simple region-based color extraction")
        
        # Get color names and print results
        print("\\n🎨 COLOR ANALYSIS RESULTS:")
        print("=" * 40)
        for i, (color, percentage) in enumerate(zip(colors, percentages)):
            name = detector.get_closest_color(color)
            hex_color = f"#{color[0]:02x}{color[1]:02x}{color[2]:02x}"
            print(f"{i+1}. {name}")
            print(f"   RGB: {tuple(color)}")
            print(f"   Hex: {hex_color}")
            print(f"   Coverage: {percentage:.1f}%")
            print()
        
        return colors, percentages, image_rgb
        
    except Exception as e:
        print(f"❌ Error processing image: {e}")
        return None

def create_test_image():
    """Create a simple test image with known colors"""
    # Create a test image with colored rectangles
    test_image = np.zeros((300, 400, 3), dtype=np.uint8)
    
    # Red rectangle
    test_image[0:100, 0:200] = [255, 0, 0]
    # Green rectangle  
    test_image[0:100, 200:400] = [0, 255, 0]
    # Blue rectangle
    test_image[100:200, 0:200] = [0, 0, 255]
    # Yellow rectangle
    test_image[100:200, 200:400] = [255, 255, 0]
    # Purple rectangle
    test_image[200:300, 0:400] = [128, 0, 128]
    
    # Save test image
    test_image_bgr = cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite('test_colors.jpg', test_image_bgr)
    print("✅ Created test image: test_colors.jpg")
    
    return 'test_colors.jpg'

print("✅ Static image testing functions loaded!")
print("\\n📋 Available functions:")
print("  • test_image_colors('image.jpg') - Extract colors from image")
print("  • create_test_image() - Create a test image with known colors")


✅ Static image testing functions loaded!
\n📋 Available functions:
  • test_image_colors('image.jpg') - Extract colors from image
  • create_test_image() - Create a test image with known colors


In [14]:
# EXAMPLE: Test with static images

# Option 1: Create and test a sample image
print("🎨 Creating a test image with known colors...")
test_image_path = create_test_image()

# Test the created image
print("\\n🧪 Testing the created image:")
test_image_colors(test_image_path)

🎨 Creating a test image with known colors...
✅ Created test image: test_colors.jpg
\n🧪 Testing the created image:
📁 Loading image: test_colors.jpg
✅ Image loaded - Size: 400x300
🎨 Used K-means clustering for color extraction
\n🎨 COLOR ANALYSIS RESULTS:
1. Purple
   RGB: (np.int64(128), np.int64(0), np.int64(127))
   Hex: #80007f
   Coverage: 33.0%

2. Red
   RGB: (np.int64(253), np.int64(0), np.int64(0))
   Hex: #fd0000
   Coverage: 17.2%

3. Yellow
   RGB: (np.int64(253), np.int64(254), np.int64(3))
   Hex: #fdfe03
   Coverage: 16.9%

4. Green
   RGB: (np.int64(0), np.int64(254), np.int64(1))
   Hex: #00fe01
   Coverage: 16.6%

5. Blue
   RGB: (np.int64(2), np.int64(0), np.int64(249))
   Hex: #0200f9
   Coverage: 16.4%



([array([128,   0, 127]),
  array([253,   0,   0]),
  array([253, 254,   3]),
  array([  0, 254,   1]),
  array([  2,   0, 249])],
 [np.float64(33.040000000000006),
  np.float64(17.16),
  np.float64(16.86),
  np.float64(16.56),
  np.float64(16.38)],
 array([[[254,   0,   0],
         [254,   0,   0],
         [254,   0,   0],
         ...,
         [  0, 255,   1],
         [  0, 255,   1],
         [  0, 255,   1]],
 
        [[254,   0,   0],
         [254,   0,   0],
         [254,   0,   0],
         ...,
         [  0, 255,   1],
         [  0, 255,   1],
         [  0, 255,   1]],
 
        [[254,   0,   0],
         [254,   0,   0],
         [254,   0,   0],
         ...,
         [  0, 255,   1],
         [  0, 255,   1],
         [  0, 255,   1]],
 
        ...,
 
        [[129,   0, 127],
         [129,   0, 127],
         [129,   0, 127],
         ...,
         [129,   0, 127],
         [129,   0, 127],
         [129,   0, 127]],
 
        [[129,   0, 127],
         [129,   

In [15]:
test_image_colors("Solid_red.svg.png")

📁 Loading image: Solid_red.svg.png
✅ Image loaded - Size: 2048x2048
🎨 Used K-means clustering for color extraction
\n🎨 COLOR ANALYSIS RESULTS:
1. Red
   RGB: (np.int64(255), np.int64(0), np.int64(0))
   Hex: #ff0000
   Coverage: 100.0%

2. Red
   RGB: (np.int64(255), np.int64(0), np.int64(0))
   Hex: #ff0000
   Coverage: 0.0%

3. Red
   RGB: (np.int64(255), np.int64(0), np.int64(0))
   Hex: #ff0000
   Coverage: 0.0%

4. Red
   RGB: (np.int64(255), np.int64(0), np.int64(0))
   Hex: #ff0000
   Coverage: 0.0%

5. Red
   RGB: (np.int64(255), np.int64(0), np.int64(0))
   Hex: #ff0000
   Coverage: 0.0%



([array([255,   0,   0]),
  array([255,   0,   0]),
  array([255,   0,   0]),
  array([255,   0,   0]),
  array([255,   0,   0])],
 [np.float64(100.0),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0)],
 array([[[255,   0,   0],
         [255,   0,   0],
         [255,   0,   0],
         ...,
         [255,   0,   0],
         [255,   0,   0],
         [255,   0,   0]],
 
        [[255,   0,   0],
         [255,   0,   0],
         [255,   0,   0],
         ...,
         [255,   0,   0],
         [255,   0,   0],
         [255,   0,   0]],
 
        [[255,   0,   0],
         [255,   0,   0],
         [255,   0,   0],
         ...,
         [255,   0,   0],
         [255,   0,   0],
         [255,   0,   0]],
 
        ...,
 
        [[255,   0,   0],
         [255,   0,   0],
         [255,   0,   0],
         ...,
         [255,   0,   0],
         [255,   0,   0],
         [255,   0,   0]],
 
        [[255,   0,   0],
         [255,   0,   0],
         [25

In [16]:
test_image_colors("blue.png")

📁 Loading image: blue.png
✅ Image loaded - Size: 251x201
🎨 Used K-means clustering for color extraction
\n🎨 COLOR ANALYSIS RESULTS:
1. Blue
   RGB: (np.int64(6), np.int64(89), np.int64(195))
   Hex: #0659c3
   Coverage: 78.8%

2. Blue
   RGB: (np.int64(5), np.int64(88), np.int64(194))
   Hex: #0558c2
   Coverage: 18.5%

3. Blue
   RGB: (np.int64(5), np.int64(88), np.int64(193))
   Hex: #0558c1
   Coverage: 2.8%

4. Blue
   RGB: (np.int64(6), np.int64(89), np.int64(195))
   Hex: #0659c3
   Coverage: 0.0%

5. Blue
   RGB: (np.int64(6), np.int64(89), np.int64(195))
   Hex: #0659c3
   Coverage: 0.0%



([array([  6,  89, 195]),
  array([  5,  88, 194]),
  array([  5,  88, 193]),
  array([  6,  89, 195]),
  array([  6,  89, 195])],
 [np.float64(78.75999999999999),
  np.float64(18.48),
  np.float64(2.76),
  np.float64(0.0),
  np.float64(0.0)],
 array([[[  6,  89, 195],
         [  6,  89, 195],
         [  6,  89, 195],
         ...,
         [  6,  89, 195],
         [  6,  89, 195],
         [  6,  89, 195]],
 
        [[  6,  89, 195],
         [  6,  89, 195],
         [  6,  89, 195],
         ...,
         [  6,  89, 195],
         [  6,  89, 195],
         [  6,  89, 195]],
 
        [[  6,  89, 195],
         [  6,  89, 195],
         [  6,  89, 195],
         ...,
         [  6,  89, 195],
         [  6,  89, 195],
         [  6,  89, 195]],
 
        ...,
 
        [[  6,  89, 195],
         [  6,  89, 195],
         [  6,  89, 195],
         ...,
         [  6,  89, 195],
         [  6,  89, 195],
         [  6,  89, 195]],
 
        [[  6,  89, 195],
         [  6,  89, 195